## Populate FASTianF1 RDF database

This notebook reports the main steps to download CSV files, process them and create an RDF dataset from them accordingly to an ontology.

To measure execution time in Jupyter notebooks: <code>pip install ipython-autotime</code>

In [1]:
# required libraries
import pandas as pd
import os
from pathlib import Path

In [2]:
# Load the required libraries
from rdflib import Graph, Literal, RDF, URIRef, Namespace
# rdflib knows about some namespaces, like FOAF
from rdflib.namespace import FOAF, XSD

In [3]:
# CHECK DATE 
import datetime

# Paths

In [4]:
# parameters and URLs
path = str(Path(os.path.abspath(os.getcwd())).parent.absolute())

circuitsUrl = path + '\FASTianF1\data\DatasetF1\circuits.csv'
constructor_resultsUrl = path + '\FASTianF1\data\DatasetF1\constructor_results.csv'
constructor_standingsUrl = path + '\FASTianF1\data\DatasetF1\constructor_standings.csv'
constructorsUrl = path + '\FASTianF1\data\DatasetF1\constructors.csv'
driver_standingsUrl = path + '\FASTianF1\data\DatasetF1\driver_standings.csv'
driversUrl = path + '\FASTianF1\data\DatasetF1\drivers.csv'
lap_timesUrl = path + '\FASTianF1\data\DatasetF1\lap_times.csv'
pit_stopsUrl = path + '\FASTianF1\data\DatasetF1\pit_stops.csv'
qualifyingUrl = path + '\FASTianF1\data\DatasetF1\qualifying.csv'
racesUrl = path + '\FASTianF1\data\DatasetF1\\races.csv'
resultsUrl = path + '\FASTianF1\data\DatasetF1\\results.csv'
sprint_resultsUrl = path + '\FASTianF1\data\DatasetF1\sprint_results.csv'
statusUrl = path + '\FASTianF1\data\DatasetF1\status.csv'
ratingsUrl = path + '\FASTianF1\data\DatasetF1\\ratings.csv'

# country codes and nationalities conversion
countriesURL = path + '\FASTianF1\data\countryCodes\wikipedia-iso-country-codes.csv'
nationalitiesURL = path + '\FASTianF1\data\countryCodes\\nationalities.csv'

# saving folder
savePath =  path + '\FASTianF1\data\\rdf\\'

# Namespaces

In [5]:
# Construct the country and the movie ontology namespaces not known by RDFlib
CNS = Namespace("http://eulersharp.sourceforge.net/2003/03swap/countries#")
FO = Namespace("https://www.dei.unipd.it/db2/groupProject/FASTianF1#")

# Countries

In [6]:
#load the country codes
# we need to convert NaN values to something else otherwise NA strings are converted to NaN -> problem with Namibia
countries = pd.read_csv(countriesURL, sep=',', index_col='English short name lower case', keep_default_na=False, na_values=['_'])

# Drivers

In [7]:
# Load the CSV files in memory
drivers = pd.read_csv(driversUrl, sep=',', index_col='driverId')
nationalities = pd.read_csv(nationalitiesURL, sep=',', index_col='num_code')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [8]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [9]:
%%time 
#measure execution time

#iterate over the drivers dataframe
for index, row in drivers.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + "driver" + the driver id as URI
    Driver = URIRef(FO["driver"+str(index)])
    # Add triples using store's add() method.
    g.add((Driver, RDF.type, FO.Driver))
    g.add((Driver, FO['hasDriverRef'], Literal(row['driverRef'], datatype=XSD.string)))
    if(str(row['number']) != '\\N'):
        g.add((Driver, FO['hasDriverNumber'], Literal(row['number'], datatype=XSD.integer)))
    if(str(row['code']) != '\\N'):
        g.add((Driver, FO['hasCode'], Literal(row['code'], datatype=XSD.string)))
    g.add((Driver, FO['hasForename'], Literal(row['forename'], datatype=XSD.string)))
    g.add((Driver, FO['hasSurname'], Literal(row['surname'], datatype=XSD.string)))
    g.add((Driver, FO['hasURL'], Literal(row['url'], datatype=XSD.string)))
    
    #Check that the date has the year-month-day format otherwise print error
    try:
        datetime.datetime.strptime(str(row['dob']), '%Y-%m-%d')
        g.add((Driver, FO['hasDateOfBirth'], Literal(row['dob'], datatype=XSD.date)))
    except ValueError:
        print("Incorrect date format")

    ## handle nationality
    # there can be more than one nationality per driver
    for nationality in str(row['nationality']).split('-'):
        nationalityName = nationality.strip()
        # check if the nationality exists in the nationalities dataframe
        # str.contains() returns an array of booleans, thus we need to use the any() method
        if((nationalities['nationality'].str.contains(nationalityName, case=False)).any() == True):
            #get the country code, convert to string and get the lower case to match the country codes in the ontology
            #There are multiple countries that correspond to American nationality, then the code for Americans is manually set to "us"
            if(nationalityName != "American"):
                code = str(nationalities[nationalities['nationality'].str.contains(nationalityName, case=False)]['alpha_2_code'].values[0]).lower()
            else:
                code = "us"
            # create the RDF node for Country
            Country = URIRef(CNS[code])
            # add the edge connecting the Driver and the Country 
            g.add((Driver, FO['hasNation'], Country))

CPU times: total: 2.03 s
Wall time: 2.05 s


In [10]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'drivers.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 703 ms
Wall time: 697 ms


# Circuits

In [11]:
# Load the CSV files in memory
circuits = pd.read_csv(circuitsUrl, sep=',', index_col='circuitId')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [12]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [13]:
%%time 
#measure execution time

substitutions = {'ó': 'o', 'ü': 'u', 'ã': 'a', ' ': ''}
substitutions2 = {'UAE': 'United Arab Emirates', 'USA': 'United States', 'UK': 'United Kingdom', 'Russia': 'Russian Federation', 'Korea': 'Korea, Republic of'}

#iterate over the circuits dataframe
for index, row in circuits.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + "circuits" + the circuit id as URI
    Circuit = URIRef(FO["circuit"+str(index)])
    loc = str(row['location'])
    # substitution of special characters with standard characters
    # special characters are not allowed in URIs
    for old, new in substitutions.items():
        loc = loc.replace(old, new)
    # create the RDF node for location
    Location = URIRef(FO["location"+loc])
    # Add triples using store's add() method.
    g.add((Location, RDF.type, FO.Location))
    g.add((Circuit, RDF.type, FO.Circuit))
    g.add((Circuit, FO['hasCircuitRef'], Literal(row['circuitRef'], datatype=XSD.string)))
    g.add((Circuit, FO['hasName'], Literal(row['name'], datatype=XSD.string)))
    g.add((Circuit, FO['hasLat'], Literal(row['lat'], datatype=XSD.float)))
    g.add((Circuit, FO['hasLng'], Literal(row['lng'], datatype=XSD.float)))
    if(str(row['alt']) != '\\N'):
        g.add((Circuit, FO['hasAlt'], Literal(row['alt'], datatype=XSD.float)))
    g.add((Circuit, FO['hasURL'], Literal(row['url'], datatype=XSD.string)))
    # add the edge connecting the Circuit and the Location 
    g.add((Circuit, FO['hasLocation'], Location))

    ## handle country
    countryName = str(row['country'])
    # substitution of abbreviations in country names for full names
    for old, new in substitutions2.items():
        countryName = countryName.replace(old, new)
    # check if the country exists
    # str.contains() returns an array of booleans, thus we need to use the any() method
    if((countries.index == countryName).any() == True):
        #get the country code, convert to string and get the lower case to match the country codes in the ontology 
        code = str(countries[countries.index == countryName]['Alpha-2 code'][0]).lower()
        # create the RDF node for Country
        Country = URIRef(CNS[code])
        # add the edge connecting the Location and the Country 
        g.add((Location, FO['hasCountry'], Country))

CPU times: total: 141 ms
Wall time: 138 ms


In [14]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'circuits.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 109 ms
Wall time: 84.7 ms


# Constructors

In [15]:
# Load the CSV files in memory
constructors = pd.read_csv(constructorsUrl, sep=',', index_col='constructorId')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [16]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [17]:
%%time 
#measure execution time

#iterate over the constructors dataframe
for index, row in constructors.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + "constructor" + the constructor id as URI
    Constructor = URIRef(FO["constructor"+str(index)])
    # Add triples using store's add() method.
    g.add((Constructor, RDF.type, FO.Constructor))
    g.add((Constructor, FO['hasConstructorRef'], Literal(row['constructorRef'], datatype=XSD.string)))
    g.add((Constructor, FO['hasName'], Literal(row['name'], datatype=XSD.string)))
    g.add((Constructor, FO['hasURL'], Literal(row['url'], datatype=XSD.string)))

    ## handle nationality
    #there can be more than one nationality per constructor
    for nationality in str(row['nationality']).split('-'):
        nationalityName = nationality.strip()
        # check if the nationality exists
        # str.contains() returns an array of booleans, thus we need to use the any() method
        if((nationalities['nationality'].str.contains(nationalityName, case=False)).any() == True):
            #get the country code, convert to string and get the lower case to match the country codes in the ontology 
            #There are multiple countries that correspond to American nationality, then the code for Americans is manually set to "us"
            if(nationalityName != "American"):
                code = str(nationalities[nationalities['nationality'].str.contains(nationalityName, case=False)]['alpha_2_code'].values[0]).lower()
            else:
                code = "us"
            # create the RDF node for country
            Country = URIRef(CNS[code])
            # add the edge connecting the Constructor and the Country 
            g.add((Constructor, FO['hasNation'], Country))

CPU times: total: 422 ms
Wall time: 431 ms


In [18]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'constructors.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 141 ms
Wall time: 145 ms


# Status

In [19]:
# Load the CSV files in memory
status = pd.read_csv(statusUrl, sep=',', index_col='statusId')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [20]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [21]:
%%time 
#measure execution time

#iterate over the status dataframe
for index, row in status.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + "status" + the status id as URI
    Status = URIRef(FO["status"+str(index)])
    # Add triples using store's add() method.
    g.add((Status, RDF.type, FO.Status))
    g.add((Status, FO['hasName'], Literal(row['status'], datatype=XSD.string)))

CPU times: total: 62.5 ms
Wall time: 51.7 ms


In [22]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'status.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 31.2 ms
Wall time: 59.5 ms


# Race (and season)

In [23]:
# Load the CSV files in memory
races = pd.read_csv(racesUrl, sep=',', index_col='raceId')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [24]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [25]:
%%time 
#measure execution time

#iterate over the races dataframe
for index, row in races.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + "race" + the race id as URI
    Race = URIRef(FO["race"+str(index)])
    # Add triples using store's add() method.
    g.add((Race, RDF.type, FO.Race))
    g.add((Race, FO['hasRound'], Literal(row['round'], datatype=XSD.integer)))
    g.add((Race, FO['hasName'], Literal(row['name'], datatype=XSD.string)))
    g.add((Race, FO['hasDate'], Literal(row['date'], datatype=XSD.date)))
    if(str(row['time']) != '\\N'):
        g.add((Race, FO['hasRaceTime'], Literal(row['time'], datatype=XSD.time)))
    g.add((Race, FO['hasURL'], Literal(row['url'], datatype=XSD.string)))
    
    if(str(row['fp1_date']) != '\\N'):
        g.add((Race, FO['hasFp1Date'], Literal(row['fp1_date'], datatype=XSD.date)))
    if(str(row['fp1_time']) != '\\N'):
        g.add((Race, FO['hasFp1Time'], Literal(row['fp1_time'], datatype=XSD.time)))
    if(str(row['fp2_date']) != '\\N'):
        g.add((Race, FO['hasFp2Date'], Literal(row['fp2_date'], datatype=XSD.date)))
    if(str(row['fp2_time']) != '\\N'):
        g.add((Race, FO['hasFp2Time'], Literal(row['fp2_time'], datatype=XSD.time)))
    if(str(row['fp3_date']) != '\\N'):
        g.add((Race, FO['hasFp3Date'], Literal(row['fp3_date'], datatype=XSD.date)))
    if(str(row['fp3_time']) != '\\N'):
        g.add((Race, FO['hasFp3Time'], Literal(row['fp3_time'], datatype=XSD.time)))
    
    # create the RDF node for circuit
    Circuit = URIRef(FO["circuit"+str(row['circuitId'])])
    # add the edge connecting the Race and the Circuit 
    g.add((Race, FO['hasCircuit'], Circuit))
    
    # create the RDF node for season
    Season = URIRef(FO["season"+str(row['year'])])
    # Add triples using store's add() method.
    g.add((Season, RDF.type, FO.Season))
    g.add((Season, FO['hasYear'], Literal(row['year'], datatype=XSD.integer)))
    g.add((Season, FO['hasURL'], Literal(row['url'], datatype=XSD.string)))
    # add the edge connecting the Race and the Season 
    g.add((Race, FO['inSeason'], Season))

CPU times: total: 1.02 s
Wall time: 1.03 s


In [26]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'race.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 1.14 s
Wall time: 1.17 s


# Race (partecipations)

In [27]:
# Load the CSV files in memory
r_partecipations = pd.read_csv(resultsUrl, sep=',', index_col='resultId')
driver_standings = pd.read_csv(driver_standingsUrl, sep=',', index_col="driverStandingsId")
constructor_standings = pd.read_csv(constructor_standingsUrl, sep=',', index_col="constructorStandingsId")
constructor_results = pd.read_csv(constructor_resultsUrl, sep=',', index_col="constructorResultsId")
# join is a dataframe containing the outer join of participation results (r_partecipations) 
# and driver results (driver_standings)
# raceId and driverId are used as join keys
join = r_partecipations.merge(driver_standings, how='outer', on=['raceId','driverId'], suffixes=('', 'Total')).fillna('\\N')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [28]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [29]:
%%time 
#measure execution time

#iterate over the join dataframe
for index, row in join.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + "r_partecipation" + the partecipation id as URI
    R_partecipation = URIRef(FO["r_partecipation"+str(index)])
    # Add triples using store's add() method.
    g.add((R_partecipation, RDF.type, FO.RacePartecipation))
    if(str(row['number']) != '\\N'):
        g.add((R_partecipation, FO['hasCarNumber'], Literal(row['number'], datatype=XSD.integer)))
    if(str(row['grid']) != '\\N'):
        g.add((R_partecipation, FO['hasStartingGridPosition'], Literal(int(row['grid']), datatype=XSD.integer)))
    if(str(row['position']) != '\\N'):
        g.add((R_partecipation, FO['hasPosition'], Literal(row['position'], datatype=XSD.integer)))
    if(str(row['positionText']) != '\\N'):
        g.add((R_partecipation, FO['hasPositionText'], Literal(row['positionText'], datatype=XSD.string)))
    if(str(row['positionOrder']) != '\\N'):
        g.add((R_partecipation, FO['hasPositionOrder'], Literal(int(row['positionOrder']), datatype=XSD.integer)))
    if(str(row['points']) != '\\N'):
        g.add((R_partecipation, FO['hasPoints'], Literal(int(row['points']), datatype=XSD.integer)))
    if(str(row['laps']) != '\\N'):
        g.add((R_partecipation, FO['hasLaps'], Literal(int(row['laps']), datatype=XSD.integer)))
    if(str(row['time']) != '\\N'):
        #g.add((R_partecipation, FO['hasResultTime'], Literal(row['time'], datatype=XSD.string)))
        print(index)
        if row['time'][0] == "+":
            #FIRST_TIME RETRIEVAL
            print("START FIRST")
            tmp = join[(join['raceId'] == row['raceId']) & (join['positionText'] == '1')]
            print("Error") if (tmp.shape[0]!=1) else None
            splitted_time = str(tmp.iloc[0]['time']).split('.')
            formatted_time = "00:00:00"[0:8-len(splitted_time[0])] + splitted_time[0] + "." + splitted_time[1].ljust(3,"0")
            first_time = datetime.datetime.strptime(formatted_time, "%H:%M:%S.%f")
            #DELTA RETRIEVAL
            print("START DELTA")
            time = str(row['time']).strip()[1:]
            if ':' in (time.split('.')[0]):
                time_distance = "00:00:00.000"[0:12-len(time)] + time
                (h, m, s) = time_distance.split('.')[0].split(':')
            else:
                time_distance = time
                (h, m, s) = (0,0, time_distance.split('.')[0])
            d = datetime.timedelta(hours=int(h), minutes=int(m), seconds=int(s), milliseconds=int(time_distance.split('.')[1]))
            #FIRST_TIME + DELTA
            new_time = first_time + d
            g.add((R_partecipation, FO['hasResultTime'], Literal(new_time.strftime("%H:%M:%S.%f")[:12], datatype=XSD.time)))
            if(int(row['raceId']) < 22):
                print("POSITION-->",str(row['positionText']),"FIRST TIME-->",first_time,"DELTA-->",d,"NEW TIME-->",
                      new_time.strftime("%H:%M:%S.%f")[:12])
        else:
            splitted_time = str(row['time']).split('.')
            formatted_time = "00:00:00"[0:8-len(splitted_time[0])] + splitted_time[0] + "." + splitted_time[1].ljust(3,"0")
            g.add((R_partecipation, FO['hasResultTime'], Literal(formatted_time, datatype=XSD.time)))
            if(int(row['raceId']) < 22):
                print("POSITION-->",str(row['positionText']),"FIRST TIME-->",str(row['time']))
    if(str(row['milliseconds']) != '\\N'):
        g.add((R_partecipation, FO['hasMillisecondsResultTime'], Literal(row['milliseconds'], datatype=XSD.integer)))
    if(str(row['fastestLap']) != '\\N'):
        g.add((R_partecipation, FO['hasFastestLap'], Literal(row['fastestLap'], datatype=XSD.integer)))
    if(str(row['rank']) != '\\N'):
        g.add((R_partecipation, FO['hasFastestLapRank'], Literal(row['rank'], datatype=XSD.integer)))
    if(str(row['fastestLapTime']) != '\\N'):
        g.add((R_partecipation, FO['hasFastestLapTime'], Literal("00:00:00.000"[0:12-len(str(row['fastestLapTime']))] + 
                                                                 str(row['fastestLapTime']), datatype=XSD.time)))
    if(str(row['fastestLapSpeed']) != '\\N'):
        g.add((R_partecipation, FO['hasFastestLapSpeed'], Literal(row['fastestLapSpeed'], datatype=XSD.string)))
    if(str(row['pointsTotal']) != '\\N'):
        g.add((R_partecipation, FO['hasTotalPoints'], Literal(int(row['pointsTotal']), datatype=XSD.integer)))
    if(str(row['positionTotal']) != '\\N'):
        g.add((R_partecipation, FO['hasTotalPosition'], Literal(int(row['positionTotal']), datatype=XSD.integer)))
    if(str(row['positionTextTotal']) != '\\N'):
        g.add((R_partecipation, FO['hasTotalPositionText'], Literal(row['positionTextTotal'], datatype=XSD.string)))
    if(str(row['wins']) != '\\N'):
        g.add((R_partecipation, FO['hasDriversWins'], Literal(int(row['wins']), datatype=XSD.integer)))
    
    # create the RDF node for driver
    Driver = URIRef(FO["driver"+str(row['driverId'])])
    # add the edge connecting the Partecipation and the Driver 
    g.add((R_partecipation, FO['hasDriver'], Driver))
    
    if(str(row['constructorId']) != '\\N'):
        # create the RDF node for constructor
        Constructor = URIRef(FO["constructor"+str(int(row['constructorId']))])
        # add the edge connecting the Partecipation and the Constructor 
        g.add((R_partecipation, FO['hasConstructor'], Constructor))
    
    # create the RDF node for race
    Race = URIRef(FO["race"+str(row['raceId'])])
    # add the edge connecting the Partecipation and the Race 
    g.add((R_partecipation, FO['partecipatedInRace'], Race))
    
    if(str(row['statusId']) != '\\N'):
        # create the RDF node for status
        Status = URIRef(FO["status"+str(int(row['statusId']))])
        # add the edge connecting the Partecipation and the Status 
        g.add((R_partecipation, FO['hasStatus'], Status))

#iterate over the constructor_standings dataframe
for index, row in constructor_standings.iterrows():
    # Get the rows of the join dataframe with the raceId and constructorId values matching those in the current row.
    tmp = join[(join['raceId'] == row['raceId']) & (join['constructorId'] == row['constructorId'])]
    #iterate over the rows found
    for index2, row2 in tmp.iterrows():
        # create the RDF node for racePartecipation
        R_partecipation = URIRef(FO["r_partecipation"+str(index2)])
        # Add triples using store's add() method.
        g.add((R_partecipation, FO['hasConstructorTotalPoints'], Literal(int(row['points']), datatype=XSD.integer)))
        g.add((R_partecipation, FO['hasConstructorTotalPosition'], Literal(row['position'], datatype=XSD.integer)))
        g.add((R_partecipation, FO['hasConstructorTotalPositionText'], Literal(row['positionText'], datatype=XSD.string)))
        g.add((R_partecipation, FO['hasConstructorsWins'], Literal(int(row['wins']), datatype=XSD.integer)))

#iterate over the constructor_results dataframe
for index, row in constructor_results.iterrows():
    # Get the rows of the join dataframe with the raceId and constructorId values matching those in the current row.
    tmp = join[(join['raceId'] == row['raceId']) & (join['constructorId'] == row['constructorId'])]
    #iterate over the rows found
    for index2, row2 in tmp.iterrows():
        # create the RDF node for racePartecipation
        R_partecipation = URIRef(FO["r_partecipation"+str(index2)])
        # Add triples using store's add() method.
        g.add((R_partecipation, FO['hasConstructorPoints'], Literal(int(row['points']), datatype=XSD.integer)))

0
POSITION--> 1 FIRST TIME--> 1:34:50.616
1
START FIRST
START DELTA
POSITION--> 2 FIRST TIME--> 1900-01-01 01:34:50.616000 DELTA--> 0:00:05.478000 NEW TIME--> 01:34:56.094
2
START FIRST
START DELTA
POSITION--> 3 FIRST TIME--> 1900-01-01 01:34:50.616000 DELTA--> 0:00:08.163000 NEW TIME--> 01:34:58.779
3
START FIRST
START DELTA
POSITION--> 4 FIRST TIME--> 1900-01-01 01:34:50.616000 DELTA--> 0:00:17.181000 NEW TIME--> 01:35:07.797
4
START FIRST
START DELTA
POSITION--> 5 FIRST TIME--> 1900-01-01 01:34:50.616000 DELTA--> 0:00:18.014000 NEW TIME--> 01:35:08.630
22
POSITION--> 1 FIRST TIME--> 1:31:18.555
23
START FIRST
START DELTA
POSITION--> 2 FIRST TIME--> 1900-01-01 01:31:18.555000 DELTA--> 0:00:19.570000 NEW TIME--> 01:31:38.125
24
START FIRST
START DELTA
POSITION--> 3 FIRST TIME--> 1900-01-01 01:31:18.555000 DELTA--> 0:00:38.450000 NEW TIME--> 01:31:57.005
25
START FIRST
START DELTA
POSITION--> 4 FIRST TIME--> 1900-01-01 01:31:18.555000 DELTA--> 0:00:45.832000 NEW TIME--> 01:32:04.387
26

351
START FIRST
START DELTA
352
START FIRST
START DELTA
353
START FIRST
START DELTA
354
START FIRST
START DELTA
355
START FIRST
START DELTA
356
START FIRST
START DELTA
368
369
START FIRST
START DELTA
370
START FIRST
START DELTA
371
START FIRST
START DELTA
372
START FIRST
START DELTA
373
START FIRST
START DELTA
390
391
START FIRST
START DELTA
392
START FIRST
START DELTA
393
START FIRST
START DELTA
394
START FIRST
START DELTA
395
START FIRST
START DELTA
396
START FIRST
START DELTA
397
START FIRST
START DELTA
398
START FIRST
START DELTA
399
START FIRST
START DELTA
412
413
START FIRST
START DELTA
414
START FIRST
START DELTA
415
START FIRST
START DELTA
416
START FIRST
START DELTA
417
START FIRST
START DELTA
418
START FIRST
START DELTA
419
START FIRST
START DELTA
420
START FIRST
START DELTA
421
START FIRST
START DELTA
434
435
START FIRST
START DELTA
436
START FIRST
START DELTA
437
START FIRST
START DELTA
438
START FIRST
START DELTA
439
START FIRST
START DELTA
440
START FIRST
START DELTA
456


1217
START FIRST
START DELTA
1218
1219
START FIRST
START DELTA
1220
START FIRST
START DELTA
1221
START FIRST
START DELTA
1222
START FIRST
START DELTA
1223
START FIRST
START DELTA
1236
1237
START FIRST
START DELTA
1238
START FIRST
START DELTA
1239
START FIRST
START DELTA
1240
START FIRST
START DELTA
1241
START FIRST
START DELTA
1242
START FIRST
START DELTA
1243
START FIRST
START DELTA
1254
1255
START FIRST
START DELTA
1256
START FIRST
START DELTA
1257
START FIRST
START DELTA
1258
START FIRST
START DELTA
1259
START FIRST
START DELTA
1260
START FIRST
START DELTA
1261
START FIRST
START DELTA
1262
START FIRST
START DELTA
1263
START FIRST
START DELTA
1274
1275
START FIRST
START DELTA
1276
START FIRST
START DELTA
1277
START FIRST
START DELTA
1278
START FIRST
START DELTA
1294
1295
START FIRST
START DELTA
1314
1315
START FIRST
START DELTA
1316
START FIRST
START DELTA


ValueError: too many values to unpack (expected 3)

In [30]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'race_partecipations.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 16.1 s
Wall time: 16.1 s


# Qualifying

In [31]:
# Load the CSV files in memory
q_partecipations = pd.read_csv(qualifyingUrl, sep=',', index_col='qualifyId')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [32]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [33]:
%%time 
#measure execution time

#iterate over the q_partecipations dataframe
for index, row in q_partecipations.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + "q_partecipation" + the qualifying partecipation id as URI
    Q_partecipation = URIRef(FO["q_partecipation"+str(index)])
    # Add triples using store's add() method.
    g.add((Q_partecipation, RDF.type, FO.QualifPartecipation))
    g.add((Q_partecipation, FO['hasCarNumber'], Literal(row['number'], datatype=XSD.integer)))
    g.add((Q_partecipation, FO['hasPosition'], Literal(row['position'], datatype=XSD.integer)))
    if(str(row['q1']) != '\\N'):
        g.add((Q_partecipation, FO['hasQ1Time'], Literal("00:00:00.000"[0:12-len(str(row['q1']))] + 
                                                         str(row['q1']), datatype=XSD.time)))
    if(str(row['q2']) != '\\N'):
        g.add((Q_partecipation, FO['hasQ2Time'], Literal("00:00:00.000"[0:12-len(str(row['q2']))] + 
                                                         str(row['q2']), datatype=XSD.time)))
    if(str(row['q3']) != '\\N'):
        g.add((Q_partecipation, FO['hasQ3Time'], Literal("00:00:00.000"[0:12-len(str(row['q3']))] + 
                                                         str(row['q3']), datatype=XSD.time)))

    # create the RDF node for driver
    Driver = URIRef(FO["driver"+str(row['driverId'])])
    # add the edge connecting the qualifyingPartecipation and the Driver 
    g.add((Q_partecipation, FO['hasDriver'], Driver))
    
    # create the RDF node for constructor
    Constructor = URIRef(FO["constructor"+str(row['constructorId'])])
    # add the edge connecting the Partecipation and the Constructor 
    g.add((Q_partecipation, FO['hasConstructor'], Constructor))
    
    # create the RDF node for qualifying
    Qualifying = URIRef(FO["qualifying"+str(row['raceId'])])
    # Add triples using store's add() method.
    g.add((Qualifying, RDF.type, FO.Qualifying))
    # add the edge connecting the qualifyingPartecipation and the Qualifying 
    g.add((Q_partecipation, FO['partecipatedInQualif'], Qualifying))
    
    # create the RDF node for race
    Race = URIRef(FO["race"+str(row['raceId'])])
    # add the edge connecting the Race and the Qualifying 
    g.add((Race, FO['hasA'], Qualifying))
    
    # Qualifying starting dates and times are stored in the races dataframe,
    # then we retrieve them using the matching raceId
    Q_date_time = races[races.index == row['raceId']]
    if(str(Q_date_time['quali_date'].values[0]) != "\\N"):
        g.add((Qualifying, FO['hasQualiDate'], Literal(Q_date_time['quali_date'].values[0], datatype=XSD.date)))
    if(str(Q_date_time['quali_time'].values[0]) != "\\N"):
        g.add((Qualifying, FO['hasQualiTime'], Literal(Q_date_time['quali_time'].values[0], datatype=XSD.time)))

CPU times: total: 6.08 s
Wall time: 6.08 s


In [34]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'qualifying.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 2.03 s
Wall time: 2.04 s


# Sprint

In [35]:
# Load the CSV files in memory
s_partecipations = pd.read_csv(sprint_resultsUrl, sep=',', index_col='resultId')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [36]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [37]:
%%time 
#measure execution time

#iterate over the s_partecipations dataframe
for index, row in s_partecipations.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + "s_partecipation" + the sprint partecipation id as URI
    S_partecipation = URIRef(FO["s_partecipation"+str(index)])
    # Add triples using store's add() method.
    g.add((S_partecipation, RDF.type, FO.SprintPartecipation))
    g.add((S_partecipation, FO['hasCarNumber'], Literal(row['number'], datatype=XSD.integer)))
    g.add((S_partecipation, FO['hasStartingGridPosition'], Literal(row['grid'], datatype=XSD.integer)))
    if(str(row['position']) != '\\N'):
        g.add((S_partecipation, FO['hasPosition'], Literal(row['position'], datatype=XSD.integer)))
    g.add((S_partecipation, FO['hasPositionText'], Literal(row['positionText'], datatype=XSD.string)))
    g.add((S_partecipation, FO['hasPositionOrder'], Literal(row['positionOrder'], datatype=XSD.integer)))
    g.add((S_partecipation, FO['hasPoints'], Literal(row['points'], datatype=XSD.integer)))
    g.add((S_partecipation, FO['hasLaps'], Literal(row['laps'], datatype=XSD.integer)))
    if(str(row['time']) != '\\N'):
        g.add((S_partecipation, FO['hasResultTime'], Literal(row['time'], datatype=XSD.string)))
    if(str(row['milliseconds']) != '\\N'):
        g.add((S_partecipation, FO['hasMillisecondsResultTime'], Literal(row['milliseconds'], datatype=XSD.integer)))
    if(str(row['fastestLap']) != '\\N'):
        g.add((S_partecipation, FO['hasFastestLap'], Literal(row['fastestLap'], datatype=XSD.integer)))
    if(str(row['fastestLapTime']) != '\\N'):
        g.add((S_partecipation, FO['hasFastestLapTime'], Literal("00:00:00.000"[0:12-len(str(row['fastestLapTime']))] + 
                                                                 str(row['fastestLapTime']), datatype=XSD.time)))
    
    # create the RDF node for driver
    Driver = URIRef(FO["driver"+str(row['driverId'])])
    # add the edge connecting the sprintPartecipation and the Driver
    g.add((S_partecipation, FO['hasDriver'], Driver))
    
    # create the RDF node for constructor
    Constructor = URIRef(FO["constructor"+str(row['constructorId'])])
    # add the edge connecting the sprintPartecipation and the Constructor
    g.add((S_partecipation, FO['hasConstructor'], Constructor))
    
    # create the RDF node for sprint
    Sprint = URIRef(FO["sprint"+str(row['raceId'])])
    # Add triples using store's add() method.
    g.add((Sprint, RDF.type, FO.Sprint))
    # add the edge connecting the sprintPartecipation and the Sprint 
    g.add((S_partecipation, FO['partecipatedInSprint'], Sprint))
    
    # create the RDF node for race
    Race = URIRef(FO["race"+str(row['raceId'])])
    # add the edge connecting the Race and the Sprint 
    g.add((Race, FO['hasA'], Sprint))
    
    # create the RDF node for status
    Status = URIRef(FO["status"+str(row['statusId'])])
    # add the edge connecting the sprintPartecipation and the Sprint 
    g.add((S_partecipation, FO['hasStatus'], Status))
    
    # Sprint starting dates and times are stored in the races dataframe,
    # then we retrieve them using the matching raceId
    S_date_time = races[races.index == row['raceId']]
    if(str(S_date_time['sprint_date'].values[0]) != "\\N"):
        g.add((Sprint, FO['hasSprintDate'], Literal(S_date_time['sprint_date'].values[0], datatype=XSD.date)))
    if(str(S_date_time['sprint_time'].values[0]) != "\\N"):
        g.add((Sprint, FO['hasSprintTime'], Literal(S_date_time['sprint_time'].values[0], datatype=XSD.time)))

CPU times: total: 141 ms
Wall time: 154 ms


In [38]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'sprint.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 78.1 ms
Wall time: 86.5 ms


# Laps

In [39]:
# Load the CSV files in memory
laps = pd.read_csv(lap_timesUrl, sep=',')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [40]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [41]:
%%time 
#measure execution time
# id for the Lap URI
id = 1
#iterate over the laps dataframe
for index, row in laps.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + "lap" + the lap id as URI
    Lap = URIRef(FO["lap"+str(id)])
    # Add triples using store's add() method.
    g.add((Lap, RDF.type, FO.Lap))
    g.add((Lap, FO['hasLapNumber'], Literal(row['lap'], datatype=XSD.integer)))
    g.add((Lap, FO['hasLapPosition'], Literal(row['position'], datatype=XSD.integer)))
    g.add((Lap, FO['hasLapTime'], Literal("00:00:00.000"[0:12-len(str(row['time']))] + 
                                          str(row['time']), datatype=XSD.time)))
    g.add((Lap, FO['hasMillisecondsTime'], Literal(row['milliseconds'], datatype=XSD.integer)))
    
    # Get the rows of the join dataframe with the raceId and driverId values matching those in the current row.
    tmp = join[(join['raceId'] == row['raceId']) & (join['driverId'] == row['driverId'])]
    #iterate over the rows found
    for index2, row2 in tmp.iterrows():
        # create the RDF node for racePartecipation
        R_partecipation = URIRef(FO["r_partecipation"+str(index2)])
        # add the edge connecting the racePartecipation and the Lap 
        g.add((R_partecipation, FO['hasLap'], Lap))
    
    id += 1

CPU times: total: 7min 53s
Wall time: 7min 52s


In [42]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'laps.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 1min 30s
Wall time: 1min 30s


# Pit stop

In [43]:
# Load the CSV files in memory
pit_stops = pd.read_csv(pit_stopsUrl, sep=',')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [44]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [45]:
%%time 
#measure execution time
# id for the PitStop URI
id = 1
#iterate over the pit_stops dataframe
for index, row in pit_stops.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + "stop" + the movie id as URI
    PitStop = URIRef(FO["stop"+str(id)])
    # Add triples using store's add() method.
    g.add((PitStop, RDF.type, FO.PitStop))
    g.add((PitStop, FO['hasStopNumber'], Literal(row['stop'], datatype=XSD.integer)))
    g.add((PitStop, FO['hasPitStopTimeOfDay'], Literal(str(row['time']), datatype=XSD.time)))
    g.add((PitStop, FO['hasMillisecondsTime'], Literal(row['milliseconds'], datatype=XSD.integer)))
    g.add((PitStop, FO['hasDuration'], Literal("00:00:00.000"[0:12-len(str(row['duration']))] + 
                                               str(row['duration']), datatype=XSD.time)))
    
    # Get the rows of the laps dataframe with the raceId, driverId and lap values matching those in the current row.
    tmp = laps[(laps['raceId'] == row['raceId']) & (laps['driverId'] == row['driverId']) & (laps['lap'] == row['lap'])]
    #iterate over the rows found
    for index2, row2 in tmp.iterrows():
        # create the RDF node for lap
        Lap = URIRef(FO["lap"+str(index2)])
        # add the edge connecting the PitStop and the Lap 
        g.add((PitStop, FO['hasPitStopLap'], Lap))
        
    # Get the rows of the join dataframe with the raceId and driverId values matching those in the current row.
    tmp2 = join[(join['raceId'] == row['raceId']) & (join['driverId'] == row['driverId'])]
    #iterate over the rows found
    for index2, row2 in tmp2.iterrows():
        # create the RDF node for racePartecipation
        R_partecipation = URIRef(FO["r_partecipation"+str(index2)])
        # add the edge connecting the racePartecipation and the PitStop 
        g.add((R_partecipation, FO['hasPitStop'], PitStop))
    
    id += 1

CPU times: total: 24.6 s
Wall time: 24.6 s


In [46]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'stops.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 2.08 s
Wall time: 2.07 s


# Ratings

In [47]:
# Load the CSV files in memory
ratings = pd.read_csv(ratingsUrl, sep=',')
# cast year to int. If type(year) = str --> Literal= year-01-01
# movies.astype({'year': 'int32'}).dtypes

In [48]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [49]:
%%time 
#measure execution time
# id for the Rating URI
id = 1
#iterate over the ratings dataframe
for index, row in ratings.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + "rating" + the movie id as URI
    Rating = URIRef(FO["rating"+str(id)])
    # Add triples using store's add() method.
    g.add((Rating, RDF.type, FO.Rating))
    g.add((Rating, FO['hasPeriod'], Literal(row['Period'], datatype=XSD.date)))
    g.add((Rating, FO['hasRating'], Literal(row['Rating'], datatype=XSD.integer)))
    g.add((Rating, FO['hasExperience'], Literal(row['Experience'], datatype=XSD.integer)))
    g.add((Rating, FO['hasRaceCraft'], Literal(row['Race Craft'], datatype=XSD.integer)))
    g.add((Rating, FO['hasAwareness'], Literal(row['Awareness'], datatype=XSD.integer)))
    g.add((Rating, FO['hasPace'], Literal(row['Pace'], datatype=XSD.integer)))
    g.add((Rating, FO['hasContractCost'], Literal(row['Contract Cost'], datatype=XSD.long)))
    g.add((Rating, FO['hasSalary'], Literal(row['Salary'], datatype=XSD.long)))
    g.add((Rating, FO['hasBuyout'], Literal(row['Buyout'], datatype=XSD.long)))
    
    # create the RDF node for season
    Season = URIRef(FO["year"+str(row['Year'])])
    # add the edge connecting the Rating and the Season 
    g.add((Rating, FO['inSeason'], Season))
    
    # in ratings csv names are full-names, then they are splitted in forename and surname
    name = str(row['Driver']).split(' ')
    forename = name[0].strip()
    surname = name[1].strip()
    # Get the rows of the drivers dataframe with the forename value (rating csv) contained in forename column of the current row (driver csv).
    subCsv = drivers[drivers['forename'].str.contains(forename, case=False)]
    # If exists at least one row in subCsv that contain also the surname of the rating
    if((subCsv['surname'].str.contains(surname, case=False)).any() == True):
        # Get the rows of the drivers dataframe with the forename and surname values contained in the matching column of the current row.
        dId = drivers[(drivers['forename'].str.contains(forename, case=False)) & (drivers['surname'].str.contains(surname, case=False))].index.values[0]
        # create the RDF node for driver
        Driver = URIRef(FO["driver"+str(dId)])
        # add the edge connecting the Rating and the Driver 
        g.add((Rating, FO['hasDriver'], Driver))
        
    id += 1

CPU times: total: 297 ms
Wall time: 296 ms


In [50]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'ratings.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 62.5 ms
Wall time: 51.9 ms
